# Synthetic Dialogue Generation with Multiple Agents

Before we begin, let's first set up our environment...

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    # Downloading only the "output" directory from the repository
    !git init .
    !git remote add -f origin https://github.com/Play-Your-Part/tutorials.git
    !git config core.sparseCheckout true
    !echo "output" >> .git/info/sparse-checkout
    !git pull origin main

    # Installing Ollama
    !curl -fsSL https://ollama.com/install.sh | sh
    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..

else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

> ⚠️ If you're using **Colab**, please, **restart the runtime** once everything above is installed

And let's first make sure we have the Ollama server running...

In [ ]:
# Let's start the ollama server
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 10  # Wait a bit for the server to start

Let's use the default model (gemma3:27b), in case you want to replace the default one, you can use `sdialog.config` as follows:

In [ ]:
import sdialog

# UNCOMMENT THE MODEL YOU WANT TO USE (TO OVERRIDE THE DEFAULT ONE)

# use any model from https://ollama.com/library)
# sdialog.config.set_llm("qwen2.5:14b")
# sdialog.config.set_llm("qwen2.5:1.5b")

## Role-Play Multi-Agent-based Dialogue Generation

Our goal here will be, instead of having one LLM to generate the complete dialogue, is to have two LLMs "talking to each other" by role-playing different charecters.

Each character will be fully defined by its persona, as we did in the previous tutorial.

### Persona

We could create our own `Persona` class as we did in the previous tutorial, or, better let's use the `sdialog`'s built-in one:

In [ ]:
from sdialog.personas import Persona

For now, let's only create one persona, Bob:

In [ ]:
bob_persona = Persona(
        name="Bob",
        role="great dad",
        circumstances="Your daughter will talk to you",
        background="Computer Science PhD.",
        personality="an extremely happy person that likes to help people",
)

Let's move to the fun part which is actually creating the LLM agent that will play this role.

Fortunatelly, we can simply use `sdialog`'s built-in `PersonaAgent` class to create an agent for our personas.

### Agent

In its simplest form, the `PersonaAgent` class only takes a `Persona` object and the LLM model name to use for it, and will create the LLM-based agent for us:

In [ ]:
from sdialog.personas import Agent

bob = Agent(bob_persona)

Let's talk with Bob a little bit as if we were his daughter:

In [ ]:
bob("Hi dad!")

In [ ]:
bob("I need your help with my birthday")

In [ ]:
bob("I want it to be about Lord of The Rings, do you think is possible?")

That's so cool! Bob is really playing his "great dad" role well :)

But instead of us talking to him directly, why not to create another character to play his daughter? Let's do it!

In [ ]:
alice_persona = Persona(
    name="Alice",
    role="lovely daughter",
    circumstances="Your birthday is getting closer and you are talking with your dad to organize the party."
                  "You want your party to be themed as Lord of The Rings."
)

alice = Agent(alice_persona)

Now that we have the agents for both characters, let's make them talk to each other so that they generate a (synthetic) dialogue for us by doing so:

In [ ]:
dialog = alice.talk_with(bob)
dialog.print()

Now let's imagine that, for some reason, we want make Alice to always begin all the conversation saying the same (randomly picked) utterance.

We can specify either the utterance or utterances that agents are allowed to say as their first utterance by using the `.set_first_utterances()` method as follows:

In [ ]:
alice.set_first_utterances(["Hi dad!", "Hello Dad, how are you?"])
# alice.set_first_utterances("Hi dad!")  # you can pass a single utterance too

dialog = alice.talk_with(bob)
dialog.print()

We can see that, this time, Alice first asked "how are you?" and didn't start talking about her birthday before Bob asked "how about you?".

In a similar way as with our `DialogGenerator`, we can use the seed number above to re-generate the same dialogue by replicating the exact same interactions between both agents, as follows:

In [ ]:
alice.talk_with(bob, seed=1230091940).print()

Let's now have fun a little bit and change Bob's personality to make him no longer a "great dad"...

In [ ]:
bob_persona.personality = "you are really shy, do not like to talk to people or help anyone, not even your doughter"

bad_bob = Agent(bob_persona, name="Bad Bob")

In [ ]:
alice.talk_with(bad_bob).print()

Cool, huh? Our agent was able to play his new "not so great dad" role very well :)

## Use Case: Dialogue Generation for STAR Dataset

Before we begin this section, make sure you have the STAR dataset downloaded in your system, inside the `datasets` folder:

In [ ]:
# Let's clone the STAR dataset repository
!git clone https://github.com/RasaHQ/STAR.git datasets/STAR

# Let's check that `dialogues` and `tasks` folders are inside `datasets/STAR`
!ls datasets/STAR

As we did with the previous tutorial, let's begin by importing STAR from `sdialog` and pointing it to the right path:

In [ ]:
from sdialog.datasets import STAR

STAR.set_path("datasets/STAR/")

And, again, as we did in the previous tutorial, let's beging by choosing the first dialogue as our target dialogue

In [ ]:
TARGET_DIALOG = 1

original_dialog = STAR.get_dialog(TARGET_DIALOG)
original_dialog.print()

Which has the following scenario:

In [ ]:
scenario = STAR.get_dialog_scenario(TARGET_DIALOG)
scenario

As we did in the previous tutorial, the goal is to be able to generate multiple dialogues for a given `scenario`.

Before, we only had to find a way to describe each `scenario` using natural language so that we can pass it to our `DialogGenerator`.

Likewise, now we have to find a way to create the right system and user agents for each `scenario` which in turn only involves retuning the right system and user `Persona`s for a given `scenario`.

Fortunately, we can use the built-in `STAR.get_user_persona_for_scenario(scenario)` and `STAR.get_system_persona_for_scenario(scenario)` methods to achieve this.

For instance, let's get the user persona for the `scenario` above:

In [ ]:
user_persona = STAR.get_user_persona_for_scenario(scenario)
print(user_persona)

Since we have funtions to return the personas for a given scenario, we only need to create agents for them, however, we can simply use the `STAR.get_agents_for_scenario(scenario)` to do it for us:

In [ ]:
system, user = STAR.get_agents_for_scenario(scenario)

Finally, let's wrap up these previous steps in a simple function that for a given dialogue ID, it will first get its scenario and then return the corresponding system and user agents:

In [ ]:
def get_agents_from_dialogue(dialog_id):
    scenario = STAR.get_dialog_scenario(dialog_id)
    return STAR.get_agents_for_scenario(scenario)

So that, we can get the agents for any dialogue as simple as:

In [ ]:
system, user = get_agents_from_dialogue(TARGET_DIALOG)

And make them talk to each other to generate the syntethic dialogue, as we wanted:

In [ ]:
system.dialog_with(user).print()

Curious about the actual prompt the agents are using? you can simply use the `.get_prompt()` method to take a look at it. For instance, let's see the user agent's one:

In [ ]:
print(user.prompt())

Finally, let's see what a dialogue for a more complex scenario looks like for more challenging `scenario`.

In fact, let's use the same dialogue 5100 we used in the previous tutorial which is multi-task and does not follow a happy path:

In [ ]:
STAR.get_dialog_scenario(5100)

In [ ]:
system, user = get_agents_from_dialogue(5100)
system.dialog_with(user).print()

### Saving our dialogues

Before we finish, as we did in the previous tutorial, let's generate one synthetic dialog for each happy `"doctor_followup"` dialog in STAR and save it to disk for later use.

In [ ]:
from tqdm.auto import tqdm

PATH_OUTPUT = "output/STAR/multi-agents"
path_txt = os.path.join(PATH_OUTPUT, "txt")
path_json = os.path.join(PATH_OUTPUT, "json")
os.makedirs(path_txt, exist_ok=True)
os.makedirs(path_json, exist_ok=True)

for dialog in tqdm(STAR.get_dialogs(task_name="doctor_followup", happy=True, multitask=False), desc="Dialog generation"):
    if os.path.exists(os.path.join(path_txt, f"{dialog.dialogId}.txt")):
        continue

    system, user = STAR.get_agents_from_dialogue(dialog.dialogId, model_name=MODEL_NAME)

    dialog = system.dialog_with(user, id=dialog.dialogId, seed=dialog.dialogId, keep_bar=False)
    dialog.to_file(os.path.join(path_json, f"{dialog.dialogId}.json"))
    dialog.to_file(os.path.join(path_txt, f"{dialog.dialogId}.txt"))

Finally, let's check the files were generated:

In [ ]:
%ls output/STAR/multi-agents/

## Exercise: Doctor-Patient Conversations

Can you replicate the previous tutorial's exercise but this time using the multi-agent approach?

1. Define the personas as before
2. Create the two agents and make them talk to each other!

In [ ]:
# TODO: do your magic!